In [1]:
from keras.layers import Input, Dense, Lambda, Layer, Multiply, Add
from keras.models import Model, Sequential
from keras.utils.vis_utils import plot_model
from keras import backend as K
import matplotlib.pyplot as plt
from scipy.stats import norm
import numpy as np

Using TensorFlow backend.


In [2]:
batch_size = 100
original_dim = 3072 #(32x32x3)
latent_dim = 16
intermediate_dim  = 256
epochs = 15

In [3]:
#plot_model(decoder, show_shapes=True, show_layer_names=True)

In [4]:
class KLDivergenceLayer(Layer):
    #keras layerként jelenik meg

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(KLDivergenceLayer, self).__init__(*args, **kwargs)

    def call(self, inputs):

        mu, log_var = inputs

        kl_batch = - .5 * K.sum(1 + log_var -
                                K.square(mu) -
                                K.exp(log_var), axis=-1)

        self.add_loss(K.mean(kl_batch), inputs=inputs)

        return inputs

In [5]:
x = Input(shape=(original_dim,))

h = Dense(1024, activation='relu')(x)

# rejtett réteg
h = Dense(intermediate_dim, activation='relu')(h)

# kimeneti réteg: átlag és a variancia logaritmusa
z_mu = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

z_mu, z_log_var = KLDivergenceLayer()([z_mu, z_log_var])

z_sigma = Lambda(lambda t: K.exp(.5*t))(z_log_var)

eps = Input(tensor=K.random_normal(shape=(K.shape(x)[0],latent_dim)))

z_eps = Multiply()([z_sigma,eps])
z = Add()([z_mu, z_eps])

decoder = Sequential([
    Dense(intermediate_dim, input_dim=latent_dim, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(original_dim, activation='sigmoid')
])

x_pred = decoder(z)

In [6]:
# a hibafüggvény most egyéni lesz: negatív log-likelihood

# log_pθ(x|z)
def nll(y_true, y_pred):
    """ Negative log likelihood (Bernoulli). """

    # keras.losses.binary_crossentropy gives the mean
    # over the last axis. we require the sum
    return K.sum(K.binary_crossentropy(y_true, y_pred), axis=-1)

In [7]:
vae = Model(inputs=[x,eps], outputs=x_pred)

In [8]:
vae.compile(optimizer='rmsprop',loss=nll)

In [9]:
vae.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3072)         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1024)         3146752     input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 256)          262400      dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 16)           4112        dense_2[0][0]                    
____________________________________________________________________________________________

In [10]:
from keras.datasets import mnist, cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

category = 4

x_train = x_train[y_train[:,0]==category,:,:] /255
y_train = y_train[y_train==category]
x_test = x_test[y_test[:,0]==category] /255
y_test = y_test[y_test==category]

In [11]:
x_train = x_train.reshape(-1, original_dim) /255

In [12]:
x_test = x_test.reshape(-1, original_dim) /255

In [13]:
vae.fit(x_train, x_train, shuffle=True, epochs=epochs, batch_size=batch_size, validation_data=(x_test, x_test))

Train on 5000 samples, validate on 1000 samples
Epoch 1/15


InternalError:  Blas GEMM launch failed : a.shape=(100, 3072), b.shape=(3072, 1024), m=100, n=1024, k=3072
	 [[node dense_1/MatMul (defined at C:\Users\Dani\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_keras_scratch_graph_1587]

Function call stack:
keras_scratch_graph


In [ ]:
encoder = Model(x, z_mu)
encoder.summary()

In [ ]:

z_test = encoder.predict(x_test, batch_size=batch_size)
plt.figure(figsize=(10, 8))
plt.scatter(z_test[:, 0], z_test[:, 1], c=y_test,
            alpha=.4, s=3**2, cmap='viridis')
plt.colorbar()
plt.show()

In [ ]:
decoder.summary()

In [ ]:
xz_sample = norm.ppf(np.random.rand(1,2))
#print (z_sample.shape)
z_sample=np.array([[0.9, 0.9]])
#print (z_sample.shape)
#print(z_sample)
generated_digit = decoder.predict(z_sample)
plt.imshow(generated_digit.reshape(32,28), cmap='gray')
plt.show()

In [ ]:
# digitek 2D síkon ábrázolva
n = 15  # 15x15 digit
digit_size = 28

# linearly spaced coordinates on the unit square were transformed
# through the inverse CDF (ppf) of the Gaussian to produce values
# of the latent variables z, since the prior of the latent space
# is Gaussian

z1 = norm.ppf(np.linspace(0.01, 0.99, n))
z2 = norm.ppf(np.linspace(0.01, 0.99, n))

z_grid = np.dstack(np.meshgrid(z1, z2))

x_pred_grid = decoder.predict(z_grid.reshape(n*n, latent_dim)) \
                     .reshape(n, n, digit_size, digit_size)

plt.figure(figsize=(10, 10))
plt.imshow(np.block(list(map(list, x_pred_grid))), cmap='gray')
plt.show()